# LangChain Tool Call과 Agent

###핵심 개념 정리

- Tool: LLM이 필요할 때 호출하는 외부 기능(예: 웹 검색, 계산기, DB 질의).

- Tool Call: 모델이 “이 질문은 검색이 필요하네”처럼 도구 호출 의사결정을 내려 실제로 도구를 실행하는 단계.

- Agent: “언제 어떤 도구를 쓸지” 스스로 판단해 도구 호출 → 결과 반영 → 답변 생성을 반복 실행하는 실행자.

- bind_tools: LLM에 사용할 도구들을 등록해 도구 호출 가능한 모델로 만드는 LangChain 방식.

### 단계 1. AI 모델 준비
- Google Gemini와 OpenAI GPT 같은 AI 모델을 불러옵니다.  
- 동시에 요청 횟수를 제한하는 ‘속도 제한 장치(Rate Limiter)’도 둡니다.  
  → 고객센터에 전화를 한꺼번에 너무 많이 걸리지 않게 막는 역할과 비슷합니다.

### 단계 2. 대화의 뼈대 만들기
- 사람 질문(HumanMessage), 시스템 지침(SystemMessage), AI 답변(AIMessage)을 정리해 대화 기록을 만듭니다.  
- 이렇게 하면 AI가 대화 맥락을 이해하고 친절한 톤으로 답할 수 있습니다.

### 단계 3. 프롬프트 체인 만들기
- “프롬프트”는 AI에게 지시하는 대본 같은 것입니다.  
- 이를 LLM과 연결해 체인으로 만들면, 질문이 들어올 때마다 같은 규칙으로 대답합니다.

### 단계 4. 외부 검색 도구 연결
- Tavily 검색 같은 도구를 붙여서, AI가 최신 정책이나 정보를 직접 찾아보고 답하게 합니다.  
- 예: “현재 환불 규정 알려줘” → AI가 도구를 써서 검색 후 답변.

### 단계 5. 사용자 정의 도구 추가
- 숫자를 합산하거나, 문장을 뒤집는 등 직접 만든 도구를 AI가 활용할 수 있습니다.  
- 이렇게 하면 단순 계산이나 텍스트 가공도 AI가 자동 처리합니다.

### 단계 6. 최종 워크플로우
- 사용자의 질문 → AI가 필요시 도구 호출 → 결과 반영 → 최종 답변 생성.  
- 즉, **AI + 규칙 + 검색도구 + 맞춤도구**가 합쳐져 고객센터 상담을 자동화하는 구조입니다.


In [1]:
!pip3 install -U langchain langchain_core langchain_community langchain_google_genai tavily-python
# 👉 필요한 라이브러리 설치 (LangChain + Google Gemini + Tavily 검색 툴)

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.0 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 457 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 463 kB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 396 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 5.1 MB/s eta 0:00:011
     |████████████████████████████████| 174 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 490 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 7.3 MB/s eta 0:00:01
     |████████████████████████████████|

In [4]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI
# 👉 요청 제한기(Rate Limiter)와 Gemini 모델 불러오기
# from langchain_openai import ChatOpenAI
# 👉 필요시 OpenAI 모델도 불러와 사용할 수 있음 (지금은 주석 처리)
# 고객지원 시나리오에서도 동일한 호출 제한 가정 (분당 10회)
request_throttle = InMemoryRateLimiter(
    requests_per_second=0.167,  # 👉 초당 약 0.167회 → 분당 10회 제한
    check_every_n_seconds=0.1,  # 👉 0.1초마다 요청 횟수 체크
    max_bucket_size=10,         # 👉 최대 10개까지 한 번에 처리 가능
)

# Rate Limiter를 적용한 LLM 인스턴스
cs_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",  # 👉 사용할 Gemini 모델 지정
    rate_limiter=request_tßrottle, # 👉 위에서 만든 요청 제한기 적용
)

# 간단 동작 확인용 테스트 질문(도메인: 고객지원)
cs_llm.invoke("고객 응대란 무엇인지 한 문장으로 설명해줘!")
# 👉 모델이 정상 작동하는지 테스트


AIMessage(content='고객 응대란 고객의 문의, 불만, 요청 등에 대해 기업이나 조직이 적절하고 신속하게 대응하여 고객 만족도를 높이는 모든 활동을 의미합니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-exp', 'safety_ratings': []}, id='run--d0281be2-54a9-44f9-b4c2-832ace8958b4-0', usage_metadata={'input_tokens': 15, 'output_tokens': 39, 'total_tokens': 54, 'input_token_details': {'cache_read': 0}})

### LLM이 답변하는 방법

In [5]:
# 1. HumanMessage를 이용한 방법
# LangChain에서 대화 메시지 타입 세 가지를 불러옵니다:
# - HumanMessage: 사용자의 질문
# - SystemMessage: 답변 규칙·조건
# - AIMessage: AI가 생성한 응답
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

support_system_msg = SystemMessage(content="""아래 [문의]에 대해 적절하게 답하시오.
답변은 2문장 이내로 하고, 친절하고 정확한 어조로 답하세요.""")
# 👉 시스템 메시지: "답변은 2문장 이내, 친절하게" 규칙 설정

customer_msg = HumanMessage(content="환불 절차를 간단히 설명해줘!")
# 👉 고객이 실제로 묻는 질문 입력

conversation_log = [support_system_msg, customer_msg]
# 👉 대화 기록을 리스트로 구성

cs_response = cs_llm.invoke(conversation_log)
cs_response
# 👉 대화 기록을 모델에 전달하고 응답 받기

# Chat 모델의 출력은 AIMessage라는 형식으로 변환됩니다.
print(cs_response.content)
# 👉 실제 답변 내용 출력


환불은 구매하신 제품의 반송이 완료되면 영업일 기준 3~5일 이내에 결제하신 수단으로 처리됩니다. 자세한 내용은 저희 웹사이트의 환불 정책 페이지를 참고해 주시면 감사하겠습니다.


In [6]:
# 2. ChatPromptTemplate을 이용한 방법
from langchain_core.prompts import ChatPromptTemplate
# 👉 대화 프롬프트(질문 틀) 생성 도구

# 입력 변수가 {inquiry}인 템플릿 (도메인: 고객지원)
support_prompt = ChatPromptTemplate([
    ('system', '''아래 [문의]에 답변하시오.
답변은 2문장 이내로 하고, 친절하고 정확한 어조로 답하세요.'''),
    ('user', '''[문의]: {inquiry}''')
])
# 👉 system 규칙 + user 질문을 템플릿으로 정의

support_prompt
# 👉 템플릿 객체 확인

# 프롬프트와 LLM(cs_llm)을 | 로 연결하여 체인 구성
support_chain = support_prompt | cs_llm
support_chain
# 👉 프롬프트와 모델을 연결해 하나의 체인으로 사용

cs_resp = support_chain.invoke({'inquiry': "고객 FAQ와 맞춤 스크립트의 차이가 뭐야?"})
print(cs_resp.content)
# 👉 템플릿에 질문 전달 → 답변 생성


고객 FAQ는 자주 묻는 질문에 대한 답변 모음집인 반면, 맞춤 스크립트는 특정 상황이나 고객 문의에 맞춰 미리 준비된 대화 흐름입니다. 따라서 맞춤 스크립트는 FAQ보다 더 구체적이고 상황에 맞는 답변을 제공할 수 있습니다.


In [7]:
# 입력 변수가 1개일 때는 값만 전달해도 됨
cs_resp = support_chain.invoke("교환/반품 정책이 뭐야?")
print(cs_resp.content)
# 👉 간단한 호출 방법 (문자열만 넣어도 됨)
# 참고: 외부 근거가 부족하면 환각이 발생할 수 있습니다.
# 👉 포인트: AI가 틀린 내용을 말할 수도 있음 (주의)

저희는 구매일로부터 30일 이내에 교환 및 반품을 제공합니다. 단, 제품이 원래 상태이고 모든 태그가 부착되어 있어야 합니다.


In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults  # Tavily 웹 검색 도구
# 👉 외부 검색을 위한 Tavily 툴 불러오기
# 고객지원 검색용 Tavily 인스턴스
support_search = TavilySearchResults(
    max_results=3,            # 👉 검색 최대 3개
    include_answer=True,      # 👉 요약 포함
    include_raw_content=True, # 👉 원문 포함
)

support_search  # 👉 객체 확인

# 간단 검색
cs_search_docs = support_search.invoke("교환과 환불 정책의 차이는?")
cs_search_docs
# 👉 검색 실행 (AI가 참고할 수 있는 자료 수집)

/var/folders/sd/72757lvs4fx3jpxplj70p5nh0000gn/T/ipykernel_65757/2903736447.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  support_search = TavilySearchResults(


[{'title': '17. 교환/반품/환불 정책 정의',
  'url': 'https://hyeyun133.tistory.com/55',
  'content': "- 카페24에서 반품은 반품처리중(수거전), 반품처리중(환불전)으로 구분되는데 환불 여부는 별도의 상태값으로 관리한다. 그래서 반품준비(수거전) 상태를 정의했다.\n\n- 한 가지 헷갈린게 반품완료 상태인데 아래 매뉴얼에 의하면 궁극적으로 반품완료라는 것은 환불이 완료된 상태를 의미한다. (환불전까지는 반품처리중 상태이므로)\n\nImage 2\n\n카페24 관리자 매뉴얼\n\nImage 3\n\n  \n\n2단계: 교환/반품/환불 정 책 정의하기\n\n여기서는 교환 정책이 가장 큰 이슈였다. 보통 반품 후 재주문과 프로세스가 다르지 않기 때문에 쇼핑몰에서는 교환 자체는 제한하고 주문 취소나 반품 후 재주문을 권고하는 경우가 있다. 아래 랑콤의 경우는 동일 상품의 옵션 교환도 불가하므로 가장 제한이 큰 경우라고 볼 수 있다.\n\nImage 4\n\n랑콤 교환정책 [...] Image 11\n> 마치며앞서 정의한대로 교환에는 3가지 경우의 수가 있다.\n> \n> \n> \n> 1)주문 상품 금액=교환 상품 금액:교환 완료 처리 됨\n> \n> 2)주문 상품 금액>교환 상품 금액: (주문 상품 금액-교환 상품 금액) 환불 처리 후 교환 완료 처리\n> \n> 3)주문 상품 금액<교환 상품 금액: (교환 상품 금액-주문 상품 금액) 입금 확인 완료 후 교환 완료 처리\n> \n> \n> \n> 다른 상품으로 교환하고 싶은 경우 2, 3번처럼 금액 차이(환불이냐, 추가 입금이냐)가 나는 것이 가장 큰 문제인데 그래서 보통 같은 상품의 옵션 변경만 가능하도록 제한하거나 아예 반품 후 재주문을 권유하는 경우가 많다.\n> \n> \n> \n> 위 플라이모델 쇼핑몰의 사례처럼 게시글에 직접 교환 신청한다면 소비자는\n> \n> \n> \n> 2) 주문 상품 금액>교환 상품 금액의 경우 환불받을 차액과 환불 

In [10]:
# LLM에 Tool 연결하기
support_tools = [support_search]  # 👉 사용할 툴 목록 정의
cs_llm_with_tools = cs_llm.bind_tools(support_tools)
# 👉 LLM에 검색 툴 연결 → AI가 직접 검색 가능해짐
cs_llm_with_tools  # 👉 툴이 등록된 모델 확인

RunnableBinding(bound=ChatGoogleGenerativeAI(rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x10d5ecf10>, model='models/gemini-2.0-flash-exp', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x10362a9d0>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [11]:
cs_llm_with_tools.kwargs['tools']  # 👉 등록된 툴 스키마 내부 확인


[{'type': 'function',
  'function': {'name': 'tavily_search_results_json',
   'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
   'parameters': {'properties': {'query': {'description': 'search query to look up',
      'type': 'string'}},
    'required': ['query'],
    'type': 'object'}}}]

In [12]:
support_tool_chain = support_prompt | cs_llm_with_tools

In [13]:
# 👉 프롬프트와 툴 연결 모델을 합쳐 체인 생성
tool_call_msg = support_tool_chain.invoke("고객지원에서 FAQ, 지식베이스, 스크립트 에이전트의 차이를 알려줘.")
tool_call_msg

AIMessage(content='FAQ는 일반적인 질문에 대한 답변을 제공하고, 지식베이스는 광범위한 정보를 체계적으로 정리하여 제공합니다. 스크립트 에이전트는 특정 시나리오에 따라 미리 정의된 스크립트를 사용하여 고객을 지원합니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-exp', 'safety_ratings': []}, id='run--b5b1634e-d4d7-4e48-bdde-8d28303f852f-0', usage_metadata={'input_tokens': 109, 'output_tokens': 57, 'total_tokens': 166, 'input_token_details': {'cache_read': 0}})

In [14]:
# 👉 프롬프트+툴 기반 답변 요청
tool_call_msg = cs_llm_with_tools.invoke("현재 시점의 고객센터 관련 주요 이슈를 검색해줘")
tool_call_msg

AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "\\ucd5c\\uadfc \\uace0\\uac1d\\uc13c\\ud130 \\uc8fc\\uc694 \\uc774\\uc288"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-exp', 'safety_ratings': []}, id='run--570bb2d3-a55a-4c3a-b31e-2280b5585dfb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '최근 고객센터 주요 이슈'}, 'id': 'ee67f645-65f0-44ca-b5fe-53d228a1c610', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 16, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}})

In [15]:
# 👉 프롬프트 없이 툴 연결 모델 직접 호출
tool_call_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': '최근 고객센터 주요 이슈'},
  'id': 'ee67f645-65f0-44ca-b5fe-53d228a1c610',
  'type': 'tool_call'}]

In [16]:
# 👉 툴 호출 여부 확인 및 매핑된 실행 함수 준비
support_tool_map = {'tavily_search_results_json': support_search}  # 툴 이름 → 실제 함수 매핑

if tool_call_msg.tool_calls:
    invoked_tool_name = tool_call_msg.tool_calls[0]['name']  # 호출된 툴 이름 추출
    tool_exec = support_tool_map.get(invoked_tool_name)       # 해당 이름의 실행 함수 가져오기
    print(invoked_tool_name, tool_exec)                       # 호출된 툴 이름과 함수 확인용 출력
else:
    print("❌ 툴 호출이 존재하지 않습니다.")                    # 호출이 없을 때 안내


tavily_search_results_json max_results=3 include_answer=True include_raw_content=True api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))


In [17]:
support_tool_msg = tool_exec.invoke(tool_call_msg.tool_calls[0])  # 👉 툴 실행 결과 수집


In [18]:
support_tool_msg  # 👉 검색 결과 확인

ToolMessage(content='[{"title": "[PDF] Business Focus - 2025년국내디지털금융 주요이슈", "url": "https://assets.kpmg.com/content/dam/kpmgsites/kr/pdf/2025/eri/business_focus/kpmg-kr-bf-2025%EB%85%84-%EA%B5%AD%EB%82%B4-%EB%94%94%EC%A7%80%ED%84%B8%EA%B8%88%EC%9C%B5-%EC%A3%BC%EC%9A%94-%EC%9D%B4%EC%8A%88_20250314.pdf.coredownload.inline.pdf", "content": "챗봇, 가상비서 AI 컨택센터 신원인식(얼굴, 음성) 초개인화서비스 지능형예측 AI 기반신용평가 AI 기반상품개발 이상거래탐지(FDS) 자금세탁방지(AML) AI 기반품질관리(불완전판매등) AI 기반리스크관리(운영/시장/신용) AI 기반심사(대출, 언더라이팅등) AI 기반디지털결산 지능형서류관리(계약관리등) 소프트웨어코딩 합성데이터 프론트오피스 (고객, 영업마케팅, 상품등) 미들오피스 (리스크, 컴플라이언스) 백오피스 (계약관리, 심사, 운영등) 33 © 2025 KPMG Samjong Accounting Corp., a Korea Limited Liability Company and a member firm of the KPMG global organization of independent member firms affiliated with KPMG International Limited, a private English company limited by guarantee. [...] Business Contacts 디지털금융·AI 서비스전문팀 Consulting Services 이동근 전무 T 02-2112-7587 E tongkeunlee@kr.kpmg.com 김세호 상무 T 02-2112-7879 E seihokim@kr.kpmg.com 황태영 상무 T 0

In [19]:
# 💬 LLM + Tool을 이용한 간단한 "고객지원 에이전트" 함수
def support_workflow(cs_llm, inquiry, tools=[support_search]):
    # ✅ 1. 사용할 툴들을 이름 기준으로 정리 (딕셔너리 형태)
    support_tool_map = {x.name: x for x in tools}

    # ✅ 2. LLM에 툴 연결 (이제 모델이 툴을 직접 호출할 수 있음)
    cs_llm_with_tools_local = cs_llm.bind_tools(tools)

    # ✅ 3. 사용자 질문 메시지 생성
    dialogs = [HumanMessage(content=inquiry)]
    print(f"📩 사용자 질문: {inquiry}")

    # ✅ 4. LLM이 질문을 받아 '툴이 필요할지' 판단
    tool_decision = cs_llm_with_tools_local.invoke(inquiry)# Agent 부분: LLM이 “이 질문에는 검색 툴을 써야겠다 / sum_counts를 써야겠다” 등을 판단
    dialogs.append(tool_decision)  # 대화 흐름에 모델 응답 추가

    # ✅ 5. 만약 LLM이 툴 사용을 결정했다면?
    if tool_decision.tool_calls:
        # 👉 호출할 툴 이름 꺼내기
        chosen_tool = tool_decision.tool_calls[0]['name']
        print(f"🛠️ 툴 실행 요청: {chosen_tool}")
        print("툴 호출 세부 정보:", tool_decision.tool_calls[0])

        # 👉 실제 툴 객체 가져오기
        tool_exec = support_tool_map[chosen_tool]

        # 👉 툴 실행 (LLM이 전달한 파라미터 사용)
        tool_output = tool_exec.invoke(tool_decision.tool_calls[0])

        # 👉 툴 실행 결과를 대화 흐름에 추가
        dialogs.append(tool_output)

        # ✅ 6. 툴 결과를 참고해 LLM이 최종 답변 생성
        final_response = cs_llm_with_tools_local.invoke(dialogs)
    else:
        # 🔹 툴이 필요 없는 경우, LLM의 첫 응답이 곧 최종 답변
        final_response = tool_decision

    # ✅ 7. 최종 답변 내용(content)만 반환
    return final_response.content

# 🚀 함수 실행 테스트
answer_text = support_workflow(cs_llm, "최신 유행 옷 트렌드를 검색해줘")
print("🤖 최종 답변:", answer_text)


📩 사용자 질문: 최신 유행 옷 트렌드를 검색해줘
🛠️ 툴 실행 요청: tavily_search_results_json
툴 호출 세부 정보: {'name': 'tavily_search_results_json', 'args': {'query': 'latest fashion trends'}, 'id': '8cf4ae6d-ce8b-4c94-b135-e09c9074a408', 'type': 'tool_call'}
🤖 최종 답변: 현재 패션 트렌드는 봄/여름, 가을/겨울 컬렉션을 중심으로 유행이 빠르게 변화하고 있습니다. 몇 가지 주요 트렌드는 다음과 같습니다.

*   **유니폼 스타일:** 정형화된 스타일에서 벗어나 자유로운 믹스 앤 매치를 시도하는 경향이 있습니다.
*   **로코코 스타일:** 화려하고 장식적인 디테일이 강조된 스타일이 다시 떠오르고 있습니다.
*   **액세서리:** 가방, 벨트, 플립플랍 등 액세서리를 활용하여 포인트를 주는 스타일이 인기를 얻고 있습니다.
*   **주얼리:** 화려하면서도 절제된 스타일의 주얼리가 트렌드를 이끌고 있습니다.
*   **데님:** 데님 소재를 활용한 다양한 스타일이 인기를 끌고 있습니다.
*   **컬러 트렌드:** 버터 옐로우, 네이비, 그린 등 다채로운 색상이 활용되고 있습니다.

자세한 내용은 Vogue, Who What Wear 등 패션 전문 웹사이트에서 더 많은 정보를 얻을 수 있습니다.


In [20]:
# Custom Tool 만들기
from langchain_core.tools import tool  # 👉 @tool 데코레이터 불러오기

@tool
def sum_counts(open_tickets: int, solved_today: int) -> int:
    "접수 티켓 수와 당일 해결 티켓 수를 합산해 총 건수를 반환합니다."
    return open_tickets + solved_today
# 👉 간단한 합산 도구 (예: 티켓 현황 계산기)

@tool
def reverse_sentence(passage: str) -> str:
    "입력된 문장을 뒤집어 반환합니다."
    return passage[::-1]
# 👉 문자열 뒤집기 도구 (예: 텍스트 가공)

print(sum_counts.invoke({'open_tickets': 12, 'solved_today': 5}))
print(reverse_sentence.invoke({'passage': '고객 응대는 친절이 핵심'}))
# 👉 사용자 정의 툴 직접 실행해보기


17
심핵 이절친 는대응 객고


In [21]:
cs_tools = [support_search, sum_counts, reverse_sentence]
# 👉 사용할 툴들을 리스트로 묶기

cs_llm.bind_tools(cs_tools)
# 👉 LLM에 사용자 정의 툴까지 연결

inquiry_text = '문장 "교환 및 환불 안내"를 거꾸로 써줘'
# 👉 테스트 질문 준비

cs_result_text = support_workflow(cs_llm, inquiry_text, cs_tools)
cs_result_text
# 👉 워크플로우 실행 (문장 뒤집기 툴 호출)


📩 사용자 질문: 문장 "교환 및 환불 안내"를 거꾸로 써줘
🛠️ 툴 실행 요청: reverse_sentence
툴 호출 세부 정보: {'name': 'reverse_sentence', 'args': {'passage': '교환 및 환불 안내'}, 'id': 'bb89aedf-69a5-4f1a-bf4c-ebc55278d016', 'type': 'tool_call'}


'뒤집힌 문장은 "내안 불환 및 환교"입니다.'

In [22]:
inquiry_text = '7과 5를 더해줘'
# 👉 테스트 질문 준비

cs_result_text = support_workflow(cs_llm, inquiry_text, cs_tools)
cs_result_text
# 👉 워크플로우 실행 (문장 뒤집기 툴 호출)


📩 사용자 질문: 7과 5를 더해줘
🛠️ 툴 실행 요청: sum_counts
툴 호출 세부 정보: {'name': 'sum_counts', 'args': {'solved_today': 5, 'open_tickets': 7}, 'id': '951ca437-f845-4e8a-8552-039a978c45b5', 'type': 'tool_call'}


'7 더하기 5는 12입니다.'